In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('CorpusLoader').master('local[4]').config("spark.executor.memory", "8g").getOrCreate()

In [2]:
import os
from pyspark.sql.functions import split, element_at, explode, map_values, array_min, broadcast, map_from_entries, arrays_zip, array_contains, monotonically_increasing_id, array_distinct, transform, arrays_zip, size, slice, collect_list, first, map_from_arrays
from pyspark.sql.types import LongType, ArrayType, IntegerType, MapType

class CorpusLoader:

    def __init__(self, root_path, spark):
        self.__root_path = root_path
        self.__spark = spark

    def load(self):
        self.__array_df = self.__load_or_create_parquet('array.parquet', self.__create_array_df)
        self.__token_df = self.__load_or_create_parquet('token.parquet', self.__create_token_df)
        self.__contains_df = self.__load_or_create_parquet('contains.parquet', self.__create_contains_df)
        self.__data_df = self.__load_or_create_parquet('data.parquet', self.__create_data_df)

    def __load_or_create_parquet(self, name, create_function):
        parquet_path = os.path.join(os.path.join(self.__root_path, 'parquets'), name)
        
        if not os.path.exists(parquet_path):
            print(f'File "{name}" not found. \n\t -- Creating "{name}" ...')
            
            df = create_function()
            df.write.parquet(parquet_path)

            print('\t -- Done.')

        print(f'Loading "{name}" ...')
        return self.__spark.read.parquet(parquet_path)

    def __create_token_df(self):
        one_gram_path = os.path.join(self.__root_path, '1')

        one_gram_df = spark.read.csv(one_gram_path, sep='\n').withColumnRenamed('_c0', 'Input')
        token_df = one_gram_df \
                .select(split('Input', '\t').alias('SplitInput')) \
                .select(element_at('SplitInput', 1).alias('Tokens')) \
                .select(explode(split('Tokens', ' ')).alias('Token')) \
                .orderBy('Token') \
                .withColumn('TokenId', monotonically_increasing_id()) 
        
        return token_df

    def __create_array_df(self):
        n_gram_directories = [os.path.join(self.__root_path, x) for x in os.listdir(self.__root_path) if x.isdigit()]
        
        input_df = None

        for path in n_gram_directories:
            new_input_df = spark.read.csv(path, sep='\n').withColumnRenamed('_c0', 'Input')
            
            if input_df is None:
                input_df = new_input_df
            else:
                input_df = input_df.union(new_input_df)

        split_df = input_df \
                    .select(split('Input', '\t').alias('SplitInput')) \
                    .select(element_at('SplitInput', 1).alias('Tokens'),
                            slice('SplitInput', 2, size('SplitInput')).alias('Data')) \
                    .select(split('Tokens', ' ').alias('Tokens'), 'Data')

        array_df = split_df.select('Tokens', transform('Data', lambda d: split(d, ',')).alias('Data')) \
                    .select('Tokens', transform('Data', lambda x: x[0].cast(IntegerType())).alias('Years'),
                            transform('Data', lambda x: x[1].cast(LongType())).alias('Frequency'),
                            transform('Data', lambda x: x[2].cast(LongType())).alias('BookFrequency')) \
                    .withColumn('NgramId', monotonically_increasing_id())

        return array_df

    def __create_contains_df(self):
        n_gram_df = self.__array_df

        n_gram_to_token_id_df = n_gram_df.select('NgramId', 'Tokens') \
                .select(explode('Tokens').alias('Token'), 'NgramId') \
                .join(self.__token_df, on='Token') \
                .groupBy('NgramId').agg(collect_list('TokenId').alias('TokenIds'))

        contains_df = n_gram_to_token_id_df.select('NgramId', 'TokenIds') \
            .withColumn('IndexArray', transform('TokenIds', lambda x, i: i)) \
            .select('NgramId', arrays_zip('IndexArray', 'TokenIds').alias('TokenIds')) \
            .select('NgramId', explode('TokenIds').alias('TokenId')) \
            .select('NgramId', 'TokenId.IndexArray', 'TokenId.TokenIds') \
            .withColumnsRenamed({'IndexArray': 'Position', 'TokenIds': 'TokenId'}) \
            .orderBy('NgramId')

        return contains_df

    ## This horrific arrays to list of structs to map construct is required, because map_from_arrays zeroes everything out.
    def __create_data_df(self):
        data_df = self.__array_df.select('NgramId', 'Years', 'Frequency', 'BookFrequency')
        data_df = data_df.withColumn('FrequencyStructs', arrays_zip('Years', 'Frequency'))
        data_df = data_df.withColumn('BookFrequencyStructs', arrays_zip('Years', 'BookFrequency'))
        data_df = data_df.withColumn('FrequencyMap', map_from_entries('FrequencyStructs'))
        data_df = data_df.withColumn('BookFrequencyMap', map_from_entries('BookFrequencyStructs'))
        data_df = data_df.select('NgramId', 'FrequencyMap', 'BookFrequencyMap')

        data_df.printSchema()
        
        return data_df.withColumnsRenamed({'FrequencyMap': 'Frequency', 'BookFrequencyMap': 'BookFrequency'})

In [3]:
cl = CorpusLoader('C:/Users/bincl/BA-Thesis/Dataset/parquets_corpus/', spark)

cl.load()

Loading "array.parquet" ...
Loading "token.parquet" ...
Loading "contains.parquet" ...
Loading "data.parquet" ...


In [4]:
cl._CorpusLoader__token_df.printSchema()
cl._CorpusLoader__contains_df.printSchema()
cl._CorpusLoader__array_df.printSchema()
cl._CorpusLoader__data_df.printSchema()

root
 |-- Token: string (nullable = true)
 |-- TokenId: long (nullable = true)

root
 |-- NgramId: long (nullable = true)
 |-- Position: integer (nullable = true)
 |-- TokenId: long (nullable = true)

root
 |-- Tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Years: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- Frequency: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- BookFrequency: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- NgramId: long (nullable = true)

root
 |-- NgramId: long (nullable = true)
 |-- Frequency: map (nullable = true)
 |    |-- key: integer
 |    |-- value: long (valueContainsNull = true)
 |-- BookFrequency: map (nullable = true)
 |    |-- key: integer
 |    |-- value: long (valueContainsNull = true)



In [5]:

print(cl._CorpusLoader__array_df.head(1))
print(cl._CorpusLoader__data_df.head(1))

[Row(Tokens=['zehn', 'niedrigsten'], Years=[1879, 1904, 1910, 1922, 1927, 1932, 1941, 1956, 1963, 1970, 1972, 1973, 1974, 1980, 1981, 1983, 1985, 1988, 1992, 1994, 1996, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2008, 2009, 2010, 2011, 2012, 2013, 2015, 2016, 2018, 2019], Frequency=[2, 1, 2, 1, 3, 2, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 5, 3, 3, 16, 1, 12, 26, 2, 2, 2, 4], BookFrequency=[2, 1, 2, 1, 3, 2, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 3, 3, 9, 1, 9, 13, 2, 2, 2, 2], NgramId=2052994367488)]
[Row(NgramId=661424963584, Frequency={1920: 1, 1925: 2, 1800: 1, 1935: 1, 1937: 4, 1811: 2, 1944: 1, 1817: 1, 1819: 2, 1820: 1, 1823: 1, 1959: 5, 1960: 4, 1961: 2, 1962: 6, 1963: 1, 1964: 1, 1837: 1, 1965: 3, 1966: 2, 1839: 1, 1969: 3, 1971: 1, 1973: 1, 1974: 1, 1848: 2, 1977: 2, 1852: 1, 1853: 3, 1990: 1, 1991: 1, 1993: 3, 1996: 2, 2001: 1, 1875: 1, 1879: 2, 2011: 1, 2012: 4, 2013: 3, 2014: 2, 2015: 10, 2016: 10, 2017: 21, 189

In [6]:
cl._CorpusLoader__contains_df.show(100)

+-------+--------+-------+
|NgramId|Position|TokenId|
+-------+--------+-------+
|      0|       0|   3350|
|      1|       0|   3705|
|      2|       0|   2168|
|      3|       0|   2082|
|      4|       0|   2018|
|      5|       0|   2486|
|      6|       0|   3526|
|      7|       0|   3244|
|      8|       0|   2171|
|      9|       0|   2991|
|     10|       0|   2952|
|     11|       0|   3163|
|     12|       0|   3083|
|     13|       0|   2180|
|     14|       0|   3689|
|     15|       0|   2310|
|     16|       0|   3377|
|     17|       0|   2360|
|     18|       0|   3548|
|     19|       0|   3282|
|     20|       0|   2783|
|     21|       0|   2330|
|     22|       0|   3041|
|     23|       0|   2238|
|     24|       0|   3019|
|     25|       0|   2667|
|     26|       0|   2427|
|     27|       0|   2334|
|     28|       0|   2308|
|     29|       0|   3849|
|     30|       0|   3366|
|     31|       0|   3915|
|     32|       0|   2960|
|     33|       0|   2634|
|

In [7]:
from pyspark.sql.functions import col
#tokens_last = cl._CorpusLoader__token_df.join(cl._CorpusLoader__contains_df, on='NgramId')
result = cl._CorpusLoader__contains_df.orderBy('NgramId', 'Position').groupBy("NgramId").agg(collect_list("TokenId").alias("TokenIds"))
result = result.withColumn("TokenId1", col("TokenIds")[0]) \
               .withColumn("TokenId2", col("TokenIds")[1])               

[Row(NgramId=2070174347469, TokenIds=[4451, 2720], TokenId1=4451, TokenId2=2720), Row(NgramId=2070174347474, TokenIds=[4499, 14084], TokenId1=4499, TokenId2=14084), Row(NgramId=2070174347484, TokenIds=[4466, 12819], TokenId1=4466, TokenId2=12819), Row(NgramId=2070174347489, TokenIds=[4450, 17143], TokenId1=4450, TokenId2=17143), Row(NgramId=2070174347500, TokenIds=[4475, 12218], TokenId1=4475, TokenId2=12218), Row(NgramId=2070174347507, TokenIds=[4522, 16640], TokenId1=4522, TokenId2=16640), Row(NgramId=2070174347513, TokenIds=[4446, 19904], TokenId1=4446, TokenId2=19904), Row(NgramId=2070174347527, TokenIds=[4474, 11305], TokenId1=4474, TokenId2=11305), Row(NgramId=2070174347539, TokenIds=[4449, 13324], TokenId1=4449, TokenId2=13324), Row(NgramId=2070174347552, TokenIds=...]

In [8]:
result = result.select("NgramId","TokenId1", "TokenId2").withColumnRenamed("NgramId", "Full_NgramId").cache()  
ngram = result
result.show(100) 
     

+------------+--------+--------+
|Full_NgramId|TokenId1|TokenId2|
+------------+--------+--------+
|          26|    2427|    null|
|          29|    3849|    null|
|         474|    2131|    null|
|         964|    2181|    null|
|        1677|    2453|    null|
|        1697|    3874|    null|
|        1806|    3362|    null|
|        1950|    3257|    null|
|        2040|    1526|    null|
|        2214|    1777|    null|
|        2250|    1535|    null|
|        2453|    1180|    null|
|        2509|    1034|    null|
|        2529|    1486|    null|
|        2927|     475|    null|
|        3091|     528|    null|
|        3506|     562|    null|
|        3764|     985|    null|
|        4590|   11969|    null|
|        4823|   10976|    null|
|        4894|   10392|    null|
|  8589934658|   11569|    null|
|  8589934965|   12389|    null|
|  8589935171|   11145|    null|
|  8589935183|   11003|    null|
|  8589935298|    9701|    null|
|  8589935317|   11841|    null|
|  8589935

In [9]:
result = result.alias("token_table").join(result.alias("ngram"), (col("token_table.TokenId1") == col("ngram.TokenId1")) & (col("ngram.TokenId2").isNull()) &(col("token_table.TokenId2").isNotNull()))
result = result.select("token_table.Full_NgramId", "token_table.TokenId2","ngram.Full_NgramId").withColumnRenamed("ngram.Full_NgramId", "Left_NgramId") 
result.show(100)

#print(result.head(2))
#print(result.tail(2))

+------------+--------+------------+
|Full_NgramId|TokenId2|Full_NgramId|
+------------+--------+------------+
| 34359869940|   11890| 51539611029|
| 34359869940|   11890| 17179869375|
| 34359875049|    4142| 51539611029|
| 34359875049|    4142| 17179869375|
| 42949679309|    4253| 51539611029|
| 42949679309|    4253| 17179869375|
| 42949683987|   18011| 51539611029|
| 42949683987|   18011| 17179869375|
| 42949698626|   17140| 51539611029|
| 42949698626|   17140| 17179869375|
| 42949707893|   19364| 51539611029|
| 42949707893|   19364| 17179869375|
| 42949715752|    1511| 51539611029|
| 42949715752|    1511| 17179869375|
| 42949717469|   18067| 51539611029|
| 42949717469|   18067| 17179869375|
| 42949728631|   12426| 51539611029|
| 42949728631|   12426| 17179869375|
| 42949733723|   16882| 51539611029|
| 42949733723|   16882| 17179869375|
| 42949741933|     959| 51539611029|
| 42949741933|     959| 17179869375|
| 42949748832|    9359| 51539611029|
| 42949748832|    9359| 17179869375|
|

In [10]:
result = result.alias("token_table").join(ngram, (col("token_table.TokenId1") == col("ngram.TokenId1")) & (col("ngram.TokenId2").isNull()))
result = result.select("Full_NgramId","Left_NgramId","NgramId").withColumnRenamed("NgramId", "Right_NgramId")      
new_table = result.show(100)

AnalysisException: Column 'token_table.TokenId1' does not exist. Did you mean one of the following? [token_table.TokenId2, token_table.Full_NgramId, token_table.Full_NgramId, TokenId1, TokenId2, Full_NgramId];
'Join Inner, (('token_table.TokenId1 = 'ngram.TokenId1) AND isnull('ngram.TokenId2))
:- SubqueryAlias token_table
:  +- Project [Full_NgramId#67L, TokenId2#59L, Full_NgramId#969L]
:     +- Join Inner, (((TokenId1#55L = TokenId1#972L) AND isnull(TokenId2#971L)) AND isnotnull(TokenId2#59L))
:        :- SubqueryAlias token_table
:        :  +- Project [NgramId#14L AS Full_NgramId#67L, TokenId1#55L, TokenId2#59L]
:        :     +- Project [NgramId#14L, TokenId1#55L, TokenId2#59L]
:        :        +- Project [NgramId#14L, TokenIds#52, TokenId1#55L, TokenIds#52[1] AS TokenId2#59L]
:        :           +- Project [NgramId#14L, TokenIds#52, TokenIds#52[0] AS TokenId1#55L]
:        :              +- Aggregate [NgramId#14L], [NgramId#14L, collect_list(TokenId#16L, 0, 0) AS TokenIds#52]
: 

In [ ]:
NgramId= new_table[0]["Full_NgramId"]

data = cl._CorpusLoader__data_df.where('NgramId == ' + str(NgramId)).select('Frequency')
print(data.first()['Frequency'])

NgramId= new_table[0]["Left_NgramId"]

data = cl._CorpusLoader__data_df.where('NgramId == ' + str(NgramId)).select('Frequency')
print(data.first()['Frequency'])

NgramId= new_table[0]["Right_NgramId"]

data = cl._CorpusLoader__data_df.where('NgramId == ' + str(NgramId)).select('Frequency')
print(data.first()['Frequency'])

In [ ]:
print((result.count(), len(result.columns)))

In [ ]:
print((cl._CorpusLoader__data_df.count(), len(cl._CorpusLoader__data_df.columns)))